In [5]:
import numpy as np
import os
from PIL import Image
import cv2
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from python_modules.heath_map_builder import HeatMapBuilder
from python_modules.python_modules.image_equalizer import ImageEqualizer


def get_file_names(directory_path):

    # Get all file names in the directory
    file_names = os.listdir(directory_path)

    # Print all file names
    #for file_name in file_names:
    #    print(file_name)

    # Optionally, store the file names in a list
    file_list = [file_name for file_name in file_names]
    print(f"There are {len(file_names)} images to analyse")
    return file_list

In [6]:
#!pip install scikit-image

### Experimento

In [7]:
# Crear carpeta de salida si no existe
output_folder = "./processed_images_ooide_con_pesos_ooide"
os.makedirs(output_folder, exist_ok=True)
image_base_path = "yolov11/datasets/Ooids/train/images/"
model_path = '/Users/rodolfolobocarrasco/Documents/projects/YOLOV5/notebooks/yolov11/runs/detect/ooid/weights/best.pt'
file_names = get_file_names(image_base_path)

There are 47 images to analyse


In [8]:

correlaciones = []
for i in range(len(file_names)):
    print("--------------------------------------------------------")
    print(f"Imagen procesada numero:{i}")
    print("--------------------------------------------------------")
    
    image_path = image_base_path+file_names[i]

    ### Heatmap Builder
    builder_obj = HeatMapBuilder(model_path)
    try:
        builder_obj.load_image(image_path)

        builder_obj.get_heat_map(heath_map_file_name=f"./images/heatmap_{image_path}.jpg")
        heat_map = builder_obj.raw_heat_map

        ## Original Image
        
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        original_image_gray = np.array(image)


        # Convert image to binary
        binary_image = original_image_gray > 64  # Threshold to create binary image

        # Step 1: Threshold the Heatmap
        threshold = np.percentile(heat_map, 70)  # Top 10% brighter zones
        binary_heatmap = (heat_map > threshold).astype(np.uint8)  # Binarize the heatmap

        # Step 2: Perform Connected Components Analysis
        num_labels, labels = cv2.connectedComponents(binary_heatmap)

        # Step 3: Filter Regions (Optional)
        region_sizes = [(label, np.sum(labels == label)) for label in range(1, num_labels)]
        large_regions = [label for label, size in region_sizes if size > 10]  # Adjust size threshold as needed
        filtered_labels = np.isin(labels, large_regions).astype(np.uint8)

        # Step 4: Save the Connected Regions Visualization
        plt.figure(figsize=(10, 10))
        plt.imshow(filtered_labels, cmap='jet')
        plt.title(f"Connected Regions of Heatmap Brighter Zones - Image {i+1}")
        plt.axis('off')  # Remove axes for a cleaner image
        plt.savefig(os.path.join(output_folder, f"connected_regions_image_{i+1}.jpg"), bbox_inches='tight', dpi=300)
        plt.close()  # Close figure to avoid memory issues

        # Step 5: Extract Connected Regions from Binary Image
        num_labels_binary, labels_binary = cv2.connectedComponents(binary_image.astype(np.uint8))
        region_sizes_binary = [(label, np.sum(labels_binary == label)) for label in range(1, num_labels_binary)]
        large_regions_binary = [label for label, size in region_sizes_binary if size > 50]
        filtered_labels_binary = np.isin(labels_binary, large_regions_binary).astype(np.uint8)

        # Confirm both masks have the same dimensions
        assert filtered_labels.shape == filtered_labels_binary.shape, "Masks must have the same shape!"

        # Step 6: Flatten Both Masks and Normalize
        heatmap_region_values = filtered_labels.flatten()
        binary_region_values = filtered_labels_binary.flatten()
        heatmap_region_values_norm = (heatmap_region_values - np.min(heatmap_region_values)) / (
            np.max(heatmap_region_values) - np.min(heatmap_region_values)
        )
        binary_region_values_norm = (binary_region_values - np.min(binary_region_values)) / (
            np.max(binary_region_values) - np.min(binary_region_values)
        )

        # Calculate Pearson Correlation
        from scipy.stats import pearsonr
        correlation, _ = pearsonr(heatmap_region_values_norm, binary_region_values_norm)
        correlaciones.append(correlation)
        print(f"Correlation between connected regions of heatmap and binary image: {correlation}")

        # Step 7: Save the Overlay of Both Masks
        plt.figure(figsize=(10, 10))
        plt.imshow(filtered_labels_binary, cmap="gray", alpha=0.5)
        plt.imshow(filtered_labels, cmap="jet", alpha=0.5)
        plt.title(f"Overlay of Connected Regions - Image {i+1}")
        plt.axis('off')  # Remove axes for a cleaner image
        plt.savefig(os.path.join(output_folder, f"overlay_image_{i+1}.jpg"), bbox_inches='tight', dpi=300)
        plt.close()  # Close figure to avoid memory issues

        print("--------------------------------------------------------")
    except Exception as e: 
        print(f"algo fallo: {e}")
        continue


--------------------------------------------------------
Imagen procesada numero:0
--------------------------------------------------------
The Yolo model was successfully loaded
Image successfully loaded

0: 1920x2560 3 Ooids, 562.7ms
Speed: 0.0ms preprocess, 562.7ms inference, 15.3ms postprocess per image at shape (1, 3, 1920, 2560)
Generating the heath map ... 
Saved combined heatmap at heatmap_combined.jpg
Correlation between connected regions of heatmap and binary image: 0.02382587076155839
--------------------------------------------------------
--------------------------------------------------------
Imagen procesada numero:1
--------------------------------------------------------
The Yolo model was successfully loaded
Image successfully loaded

0: 1920x2560 2 Ooids, 470.0ms
Speed: 0.0ms preprocess, 470.0ms inference, 18.6ms postprocess per image at shape (1, 3, 1920, 2560)
Generating the heath map ... 
Saved combined heatmap at heatmap_combined.jpg
Correlation between connecte